# Свёртки и CNN на практике

Этот ноутбук — практическая часть занятия по свёрточным нейросетям. Здесь ты:
- наглядно увидишь, как **ядра свёртки** меняют изображение (kernel playground);
- обучишь простую **CNN на MNIST**;
- улучшишь модель с помощью **data augmentation**.

Предполагается, что ты уже знаком с Python, numpy и базами PyTorch (Tensor, nn.Module, optim).

---

## Напомним, зачем вообще мы это делаем

**Свёртка** — это операция, суть которой — «пройти по изображению маленьким окном (ядром) и в каждой точке посчитать взвешенную сумму пикселей».

Так сеть не смотрит на всё изображение сразу, а ищет локальные паттерны (края, текстуры), и ядро **обучается** под задачу. В отличие от полносвязного слоя, количество параметров мало и не зависит от размера картинки — поэтому CNN масштабируются на изображения высокого разрешения.

---

## 0. Импорты и настройки



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

---

## 1. Kernel playground: как свёртки меняют изображение

Начнём с самого интуитивного — применим **разные ядра свёртки** к изображению и посмотрим, что происходит.

**Что такое ядро?** Ядро — это маленькая матрица весов (например 3×3). Свёртка в каждой точке картинки: накладываем ядро, перемножаем поэлементно и суммируем — получаем одно число. Сдвигаем окно — получаем карту признаков (соответствующих ядру). Разные ядра выделяют разное: размытие (усреднение), резкость, границы по вертикали/горизонтали.

### 1.1 Загрузка изображения

Сначала возьмём пример из MNIST. Ниже можно будет загрузить **своё фото** и применить к нему те же ядра.

In [ ]:
sample_dataset = datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

img, label = sample_dataset[0]
img_np = img.squeeze().numpy()

plt.imshow(img_np, cmap="gray")
plt.title(f"Label: {label}")
plt.axis("off")

### 1.1.1 Загрузить своё фото (опционально)

Укажи путь к своему изображению в переменной `photo_path` ниже (или оставь `None`). Изображение будет конвертировано в градации серого и приведено к размеру 28×28, затем подставлено вместо цифры из MNIST. Дальше все ядра применятся уже к твоей картинке.

In [ ]:
from PIL import Image

# Укажи путь к своему фото (например: "C:/Users/User/photo.jpg") или оставь None
photo_path = None  # или "путь/к/твоему/фото.jpg"

if photo_path:
    try:
        pil_img = Image.open(photo_path).convert("L")
        pil_img = pil_img.resize((28, 28))
        arr = np.array(pil_img, dtype=np.float32) / 255.0
        img_np = arr
        img = torch.tensor(arr, dtype=torch.float32).unsqueeze(0)  # 1, H, W
        plt.imshow(img_np, cmap="gray")
        plt.title("Твоё фото (28×28)")
        plt.axis("off")
        plt.show()
        print("Используется твоё фото!")
    except Exception as e:
        print(f"Ошибка загрузки фото: {e}")
        print("Используется изображение из MNIST выше.")
else:
    print("Путь не указан. Используется изображение из MNIST выше.")

---

### 1.2 Функция для применения ядра

Мы вручную зададим ядро и применим его через `Conv2d`. **Padding** (отступ) — добавление нулей по краям картинки перед свёрткой, чтобы размер карты на выходе совпал с входом (или не уменьшался так сильно). Здесь `padding=kernel_size//2` даёт «same»-режим: выход той же высоты и ширины.

In [ ]:
def apply_kernel(image, kernel):
    kernel = torch.tensor(kernel, dtype=torch.float32)
    kernel = kernel.unsqueeze(0).unsqueeze(0)  # [1,1,H,W]

    conv = nn.Conv2d(
        in_channels=1,
        out_channels=1,
        kernel_size=kernel.shape[-1],
        bias=False,
        padding=kernel.shape[-1] // 2
    )

    conv.weight.data = kernel
    conv.weight.requires_grad = False

    with torch.no_grad():
        output = conv(image.unsqueeze(0)).squeeze()

    return output.numpy()

---

### 1.3 Примеры ядер


In [ ]:
kernels = {
    "Blur": (1/9) * np.array([
        [1,1,1],
        [1,1,1],
        [1,1,1]
    ]),
    "Sharpen": np.array([
        [ 0,-1, 0],
        [-1, 5,-1],
        [ 0,-1, 0]
    ]),
    "Edge (Sobel X)": np.array([
        [-1,0,1],
        [-2,0,2],
        [-1,0,1]
    ]),
    "Edge (Sobel Y)": np.array([
        [-1,-2,-1],
        [ 0, 0, 0],
        [ 1, 2, 1]
    ]),
    "Emboss": np.array([
        [-2,-1,0],
        [-1, 1,1],
        [ 0, 1,2]
    ])
}

In [ ]:
fig, axes = plt.subplots(1, len(kernels) + 1, figsize=(15,4))

axes[0].imshow(img_np, cmap="gray")
axes[0].set_title("Original")
axes[0].axis("off")

for ax, (name, kernel) in zip(axes[1:], kernels.items()):
    out = apply_kernel(img, kernel)
    ax.imshow(out, cmap="gray")
    ax.set_title(name)
    ax.axis("off")

plt.show()

👉 **Задания (kernel playground):**
1. Посмотри разницу между **Sobel X** и **Sobel Y**: одно выделяет вертикальные границы, другое — горизонтальные. Почему?
2. Попробуй изменить значения ядра, размер (5×5), padding и посмотри, как меняется результат.
3. Придумай своё ядро 3×3 (например «только диагональ» или лапласиан `[[0,-1,0],[-1,4,-1],[0,-1,0]]`) и добавь его в словарь `kernels`, затем перезапусти ячейку с графиком.

---

## 2. Подготовка данных MNIST

Данные приводятся к тензорам и **нормализуются** (вычитаем среднее по датасету, делим на std) — так градиенты ведут себя стабильнее и обучение быстрее сходится.

Для наглядности мы будем использовать **только часть обучающих данных** (10% = 6000 примеров вместо 60000). Это усложнит задачу и сделает различия между сетями более заметными.

### 2.1 DataLoader без аугментаций (baseline)


In [ ]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset_full = datasets.MNIST("./data", train=True, download=True, transform=train_transform)
test_dataset  = datasets.MNIST("./data", train=False, download=True, transform=test_transform)

# Используем только 10% данных для обучения (6000 из 60000) - усложняем задачу
train_size = int(0.1 * len(train_dataset_full))
train_dataset, _ = torch.utils.data.random_split(train_dataset_full, [train_size, len(train_dataset_full) - train_size])

print(f"Используем {len(train_dataset)} примеров для обучения (вместо {len(train_dataset_full)})")

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=1000, shuffle=False)

---

## 3. Сравнение: MLP vs CNN

Сначала создадим **MLP (полносвязную сеть)** и **CNN** с **сопоставимым числом параметров**, чтобы увидеть, какая архитектура лучше справляется с изображениями.

### 3.1 MLP (Multi-Layer Perceptron)

Полносвязная сеть: каждый пиксель соединён со всеми нейронами следующего слоя. Для изображения 28×28 это 784 входных нейрона. MLP не учитывает пространственную структуру — для неё это просто вектор из 784 чисел.

In [ ]:
class SimpleMLP(nn.Module):
    """Полносвязная сеть с числом параметров ~45k (сопоставимо с CNN)"""
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 55)
        self.fc2 = nn.Linear(55, 30)
        self.fc3 = nn.Linear(30, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # [batch, 784]
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)



### 3.2 CNN (Convolutional Neural Network)

Типичная схема: несколько блоков **Conv → ReLU → MaxPool**, затем **полносвязные слои**. **MaxPool(2,2)** берёт максимум в окне 2×2 и уменьшает карту в 2 раза — так мы снижаем размерность и даём сети смотреть на более крупные области. CNN учитывает локальные паттерны (края, текстуры) благодаря свёрткам. В конце получаем 10 логитов по классам (цифры 0–9).

In [ ]:
class SimpleCNN(nn.Module):
    """Свёрточная сеть с числом параметров ~44k (сопоставимо с MLP)"""
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


# Посчитаем число параметров
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

mlp = SimpleMLP()
cnn = SimpleCNN()
print(f"MLP параметров: {count_parameters(mlp):,}")
print(f"CNN параметров: {count_parameters(cnn):,}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

## 4. Обучение baseline-модели

### Train/Eval loop

Давайте разберемся со схемой обучения и валидации которая подойдет для 90% задач в будущем.

```python
for epoch in range(max_epochs):  # <--------------------- итерируемся по всему датасету несколько раз
    
    # ОБУЧЕНИЕ
    model.train() # <------------------------------------ переводим модель в режим train
    for x_batch, y_batch in train_dataloader:  # <- ----- итерируемся по датасету. Так как мы используем SGD а не GD,
                                               #          то берем батчи заданного размера 
        x_batch, y_batch = x_batch.to(device), y_batch.to(device) # <- переносим данные на устройство (если используем GPU)
        optimizer.zero_grad()  # <----------------------- обнуляем градиенты модели
        out = model(x_batch)
        loss = criterion(out, y_batch)  # <-------------- считаем лосс
        loss.backward() # <------------------------------ считаем градиенты
        optimizer.step() # <----------------------------- делаем шаг градиентного спуска

    # ВАЛИДАЦИЯ
    model.eval() # <------------------------------------ переводим модель в режим eval
    with torch.no_grad(): # <--------------------------- НЕ считаем градиенты
        for x_batch, y_batch in val_dataloader: # <----- итерации по датасету
            x_batch, y_batch = x_batch.to(device), y_batch.to(device) # <- снова на устройство 
            out = model(x_batch)
            preds = out.argmax(dim=1) # <--------------- выбираем ответом модели индекс максимального нейрона на выходе
    compute_metrics(preds, y_batch) # <----------------- считаем метрики. У нас на практике будет подсчёт accuracy на ходу
        
```

### Задание. Дополните циклы обучения и валидации

Здесь мы реализуем циклы для одной эпохи, но вообще это не обязательно.
Можно объединить трейн и валидацию в одну функцию, оформить весь цикл выше в функцию, принимающую число эпох. Или не пользоваться функциями и переписывать каждый раз код обучения заново. Главное, чтобы работало, а остальное - вопрос удобства :)

In [ ]:
def train_one_epoch(model, train_dataloader, optimizer):
    # ВАШ КОД: переведите модель в режим train
    
    total_loss = 0 # для отследивания прогресса 
    for x_batch, y_batch in train_dataloader:
        # ВАШ КОД: перенесите данные на устройство
        
        # ВАШ КОД: вычислите выходы модели, посчитайте лосс. НЕ ЗАБУДЬТЕ zero_grad!
        
        total_loss += loss.item()   # прибавляем лосс батча к общему
    return total_loss / len(train_dataloader) # возвращаем средний лосс


def evaluate(model, val_dataloader):
    # ВАШ КОД: переведите модель в режим eval
    
    correct = 0 # будем считать accuracy "на ходу"
    total = 0
    with torch.no_grad():
        for x_batch, y_batch in val_dataloader:
            # ВАШ КОД: перенесите данные на устройство, вычислите выход модели
            x_batch, y_batch = 
            out = 
            
            # Выбираем меткой класса индекс максимального элемента
            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item() # считаем правильные ответы
            total += y.size(0) # общее число ответов
    return correct / total # возвращаем accuracy

In [ ]:
# Обучаем MLP
print("=" * 50)
print("Обучение MLP:")
print("=" * 50)
mlp_model = SimpleMLP().to(device)
mlp_optimizer = optim.Adam(mlp_model.parameters(), lr=1e-3)
for epoch in range(3):
    loss = train_one_epoch(mlp_model, train_loader, mlp_optimizer)
    acc = evaluate(mlp_model, test_loader)
    print(f"Epoch {epoch+1}: loss={loss:.4f}, acc={acc:.4f}")

print("\n" + "=" * 50)
print("Обучение CNN:")
print("=" * 50)
cnn_model = SimpleCNN().to(device)
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=1e-3)
for epoch in range(3):
    loss = train_one_epoch(cnn_model, train_loader, cnn_optimizer)
    acc = evaluate(cnn_model, test_loader)
    print(f"Epoch {epoch+1}: loss={loss:.4f}, acc={acc:.4f}")

print("\n" + "=" * 50)
print("Итоговое сравнение:")
print("=" * 50)
mlp_final = evaluate(mlp_model, test_loader)
cnn_final = evaluate(cnn_model, test_loader)
print(f"MLP accuracy: {mlp_final:.4f}")
print(f"CNN accuracy: {cnn_final:.4f}")
print(f"Разница: {cnn_final - mlp_final:.4f} ({((cnn_final - mlp_final) / mlp_final * 100):.1f}% лучше)")

In [ ]:
# Предсказание для одного примера (MLP vs CNN)
x, y = next(iter(test_loader))
sample, true_label = x[0:1].to(device), y[0].item()

mlp_model.eval()
cnn_model.eval()
with torch.no_grad():
    mlp_logits = mlp_model(sample)
    cnn_logits = cnn_model(sample)
mlp_pred = mlp_logits.argmax(dim=1).item()
cnn_pred = cnn_logits.argmax(dim=1).item()

plt.imshow(x[0].squeeze().numpy(), cmap="gray")
plt.title(f"Истина: {true_label}\nMLP: {mlp_pred} {'✓' if mlp_pred == true_label else '✗'}\nCNN: {cnn_pred} {'✓' if cnn_pred == true_label else '✗'}")
plt.axis("off")
plt.show()

# Визуализация: как выглядит изображение после обработки каждым фильтром conv1
print("=" * 60)
print("Визуализация активаций: что 'видит' каждый фильтр conv1")
print("=" * 60)

cnn_model.eval()
with torch.no_grad():
    # Применяем только conv1 (без pooling и ReLU для наглядности)
    conv1_out = cnn_model.conv1(sample.to(device))  # [1, 6, H, W]
    conv1_out = conv1_out.cpu().numpy()[0]  # [6, H, W]

# Показываем оригинал и результаты каждого фильтра
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
axes[0, 0].imshow(x[0].squeeze().numpy(), cmap="gray")
axes[0, 0].set_title("Оригинал")
axes[0, 0].axis("off")

for i in range(6):
    row = (i + 1) // 4
    col = (i + 1) % 4
    if row < 2:
        # Нормализуем активации для визуализации
        activation = conv1_out[i]
        activation_norm = (activation - activation.min()) / (activation.max() - activation.min() + 1e-8)
        axes[row, col].imshow(activation_norm, cmap="gray")
        axes[row, col].set_title(f"Фильтр {i+1}")
        axes[row, col].axis("off")

# Скрываем последнюю ячейку если фильтров 6
if 6 < 7:
    axes[1, 3].axis("off")

plt.suptitle("Как выглядит изображение после каждого фильтра conv1", fontsize=12)
plt.tight_layout()
plt.show()

# Визуализация: что 'видит' второй свёрточный слой (после первого слоя + pooling)
print("\n" + "=" * 60)
print("Визуализация активаций conv2: что 'видит' второй слой")
print("=" * 60)

with torch.no_grad():
    # Проходим через conv1 + pooling + ReLU
    x_after_conv1 = F.relu(cnn_model.conv1(sample.to(device)))
    x_after_pool1 = cnn_model.pool(x_after_conv1)  # [1, 6, H, W]
    
    # Применяем conv2 к каждому каналу отдельно (для визуализации)
    # Берем первые 8 фильтров conv2
    conv2_out = cnn_model.conv2(x_after_pool1).cpu().numpy()[0]  # [16, H, W]

fig, axes = plt.subplots(2, 5, figsize=(14, 6))
# Показываем один из каналов после conv1+pool
axes[0, 0].imshow(x_after_pool1[0, 0].cpu().numpy(), cmap="gray")
axes[0, 0].set_title("После conv1+pool\n(канал 0)")
axes[0, 0].axis("off")

for i in range(8):
    row = (i + 1) // 5
    col = (i + 1) % 5
    if row < 2:
        activation = conv2_out[i]
        activation_norm = (activation - activation.min()) / (activation.max() - activation.min() + 1e-8)
        axes[row, col].imshow(activation_norm, cmap="gray")
        axes[row, col].set_title(f"Фильтр {i+1}")
        axes[row, col].axis("off")

plt.suptitle("Активации conv2: что 'видит' второй свёрточный слой (первые 8 из 16)", fontsize=11)
plt.tight_layout()
plt.show()

В чём разница между фильтрами разных слоёв? Что выделяет (белым цветом) каждый из них? Какой из слоёв несёт более общую информацию?

---

## 5. Data augmentation

**Data augmentation** — это искусственное расширение выборки: каждую эпоху одну и ту же картинку показываем модели в разных вариантах (поворот, сдвиг, масштаб). Модель учится быть устойчивой к таким искажениям и лучше обобщает. Для лиц это особенно полезно: повороты головы, сдвиги кадра.

Можно добавить больше аугментаций [отсюда](https://docs.pytorch.org/vision/2.0/transforms.html)

In [ ]:
augmented_transform = transforms.Compose([
    transforms.RandomAffine(
        degrees=15,
        translate=(0.1, 0.1),
        scale=(0.9, 1.1)
    ),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset_aug = datasets.MNIST("./data", train=True, download=True, transform=augmented_transform)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=64, shuffle=True)


👉 **Задание:** переинициализируй CNN и обучи её заново с аугментациями. Сравни точность с baseline (без аугментаций). Можно добавить новые аугментации из списка по ссылке выше

С аугментациями модель видит каждый пример в разных вариантах (повороты, сдвиги), что усложняет обучение, но улучшает обобщение. На малом количестве данных (10% от MNIST) разница должна быть заметна!

In [ ]:
cnn_aug = # ВАШ КОД ЗДЕСЬ
cnn_aug_optimizer = # ВАШ КОД ЗДЕСЬ
for epoch in range(3):
    # ВАШ КОД ЗДЕСЬ

print(f"\nБез аугментаций: {cnn_final:.4f}")
print(f"С аугментациями: {evaluate(cnn_aug, test_loader):.4f}")

## 6. Что можно посмотреть дальше

- Увеличить или уменьшить размер сети.
- Добавить [Dropout](https://ru.wikipedia.org/wiki/Исключение_(нейронные_сети)) или [BatchNorm](https://en.wikipedia.org/wiki/Batch_normalization).
- Попробовать другие аугментации.
- Посмотреть на первые слои сети и визуализировать learned-фильтры.

---

Если этот пайплайн понятен — ты готов двигаться дальше к более сложным архитектурам и задачам с лицами.